In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
train = pd.read_csv('/content/drive/MyDrive/흡연자분류/train.csv')
test = pd.read_csv('/content/drive/MyDrive/흡연자분류/test.csv')

### 모델링 이전 전처리
- 범주형 변수 인코딩
- 필요한 변수 로그 변환, 표준화 변환
- 제외할 변수 및 이유
  - 키, 몸무게, BMI: 비교적 label과 높은 상관을 띄긴하지만 이는 나이와의 상호작용 때문 (즉, 키와 몸무게값이 큰 사람들은 대부분 20~40대인데, 이들이 흡연을 많이함)
  - 시력: 낮은 상관
  - 요 단백: 상관이 낮은데, 굳이 라벨인코딩을 해서 변수량을 늘릴 필요 x

In [5]:
def encoding_model(df=None,num=0): # train=0, test=1

    if num == 0:
    # 1. 변수 선택
      model_col = [ '충치','공복 혈당', '혈압','중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈',
                    '간 효소율', 'label']

    else:
      model_col = [ '충치','공복 혈당', '혈압','중성 지방', '혈청 크레아티닌', '콜레스테롤', '고밀도지단백', '저밀도지단백', '헤모글로빈',
                   '간 효소율']

    model_df = df[model_col]

    # 2. 변수 타입 변경
    model_df = pd.get_dummies(model_df, columns=['충치'], prefix='충치')
    model_df['충치_0'] = model_df['충치_0'].astype('int64')
    model_df['충치_1'] = model_df['충치_1'].astype('int64')

    # 3. 로그 변환
    log_col = ['공복 혈당', '중성 지방', '혈청 크레아티닌', '고밀도지단백', '저밀도지단백', '간 효소율']

    model_df[log_col] = np.log1p(model_df[log_col])

    # 4. 표준화 변환
    from sklearn.preprocessing import StandardScaler
    continuous_cols = [col for col in model_df.columns if col not in [ '충치_0', '충치_1', 'label']]
    scaler = StandardScaler()
    model_df[continuous_cols] = scaler.fit_transform(model_df[continuous_cols])

    return(model_df)

In [32]:
train = encoding_model(df=train,num=0)
test = encoding_model(df=test,num=1)

### 이상치 제거
- 모델링 이전에 전처리된 데이터 내에서 label과 상관이 높은 변수의 이상치 제거

In [24]:
cor = train.corr()
cor.sort_values(by='label',ascending=False).head(2)

,공복 혈당,혈압,중성 지방,혈청 크레아티닌,콜레스테롤,고밀도지단백,저밀도지단백,헤모글로빈,간 효소율,label,충치_0,충치_1
label,0.114600,0.019119,0.253944,0.248412,-0.009176,-0.186794,-0.051707,0.402002,-0.199237,1.000000,-0.098468,0.098468
헤모글로빈,0.119496,0.045464,0.296858,0.428799,0.079272,-0.257434,0.066987,1.000000,-0.399293,0.402002,-0.076729,0.076729


- 상관관계가 0.4로 가장 높은 헤모글로빈의 이상값 검출 후 제거

In [25]:
#이상치 인덱스를  찾는 함수

def get_outlier(df=None, column=None, weight=1.5):
    # fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함.
    fraud = df[df['label']==1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    # IQR을 구하고, IQR에 1.5를 곱하여 최대값과 최소값 지점 구함.
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    # 최대값 보다 크거나, 최소값 보다 작은 값을 아웃라이어로 설정하고 DataFrame index 반환.
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index

In [33]:
outlier_index = get_outlier(df=train, column='헤모글로빈', weight=1.5)
print('이상치 데이터 인덱스:', outlier_index)

이상치 데이터 인덱스: Int64Index([ 195,  223,  308,  466,  523,  595, 1063, 1178, 1339, 1509, 1924,
            2104, 2123, 2223, 2367, 2371, 2482, 2505, 2544, 2614, 2924, 2954,
            2986, 2990, 3206, 3219, 3226, 3283, 3394, 3397, 3436, 3498, 3526,
            3718, 3736, 3801, 3845, 4059, 4288, 4373, 4498, 4696, 4925, 4997,
            5007, 5131, 5306, 5774, 5780, 5854, 5864, 5954, 5972, 6055, 6127,
            6163, 6198, 6248, 6253, 6333, 6386, 6546, 6636, 6658, 6756, 6882],
           dtype='int64')


In [34]:
def get_preprocessed_df(df=None):
    df_copy = df.copy()

    # 이상치 데이터 삭제하는 로직 추가
    outlier_index = get_outlier(df=df_copy, column='헤모글로빈', weight=1.5)
    df_copy.drop(outlier_index, axis=0, inplace=True)
    return df_copy

In [36]:
train = get_preprocessed_df(train)

### train, test 데이터 세트 반환


In [14]:
def get_train_test_dataset(df=None):

    df_copy = df.copy()
    X_features = df_copy.drop(columns='label')
    y_target = df_copy['label']

    from sklearn.model_selection import train_test_split
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 Stratified 기반 분할; stratified 방식 -> 트레인, 테스트 셋 레이블 값 분포 동일하게함
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)

    return X_train, X_test, y_train, y_test

In [37]:
X_train, X_test, y_train, y_test = get_train_test_dataset(train)

In [38]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
0    63.872363
1    36.127637
Name: label, dtype: float64
테스트 데이터 레이블 값 비율
0    63.878875
1    36.121125
Name: label, dtype: float64


- train과 test 데이터셋 비슷하게 분리됨을 확인

### 성능평가 지표 생성

In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [51]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)

### 데이터셋 오버샘플링

In [39]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (5547, 11) (5547,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (7086, 11) (7086,)
SMOTE 적용 후 레이블 값 분포: 
 1    3543
0    3543
Name: label, dtype: int64


## 모델링
- 로지스틱 회귀, LGBM 사용
- 오버샘플링된 데이터셋과 오버샘플링되지 않은 데이터셋 모두 비교
- 총 4가지 모델의 성능 비교

### 1. 로지스틱 회귀
- (1-1) 로지스틱 + 오버샘플링 X

In [52]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[714 172]
 [259 242]]
정확도: 0.6893, 정밀도: 0.5845, 재현율: 0.4830,    F1: 0.5290, AUC:0.6445


- (1-2) 로지스틱 + 오버샘플링 O

In [42]:
lr_clf = LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

오차 행렬
[[598 288]
 [132 369]]
정확도: 0.6972, 정밀도: 0.5616, 재현율: 0.7365,    F1: 0.6373, AUC:0.7057


오버샘플링을 실시했을 때, 정확도는 0.697이며 재현율 역시 0.73으로 상승함

### 2. LGBM
- (2-1) lgbm + 오버샘플링 X

In [43]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2004, number of negative: 3543
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1224
[LightGBM] [Info] Number of data points in the train set: 5547, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.361276 -> initscore=-0.569829
[LightGBM] [Info] Start training from score -0.569829
오차 행렬
[[698 188]
 [229 272]]
정확도: 0.6994, 정밀도: 0.5913, 재현율: 0.5429,    F1: 0.5661, AUC:0.6654


- (2-2) lgbm + 오버샘플링 O

In [53]:
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1)
get_model_train_eval(lgbm_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3543, number of negative: 3543
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000843 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2297
[LightGBM] [Info] Number of data points in the train set: 7086, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
오차 행렬
[[672 214]
 [212 289]]
정확도: 0.6929, 정밀도: 0.5746, 재현율: 0.5768,    F1: 0.5757, AUC:0.6677


오버샘플링 시, 정확도는 0.07 줄었지만, 재현율이 상승함

## 그리드 서치
- 4가지 모델의 결과를 보면 대부분 0.6의 정확도, 0.5대의 정밀도를 가짐
- 흡연자(1)를 정확하게 예측하는 것이 목적이므로 그리드서치를 통해 파라미터 수정 후, 모델 성능 개선 필요

### (1) 로지스틱 그리드 서치

In [81]:
from sklearn.model_selection import GridSearchCV

def lo_grid(model,x,y):
    # 그리드 서치 파라미터 그리드 정의
    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100]
    }

    # GridSearchCV를 사용하여 그리드 서치 수행
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
    grid_search.fit(x, y)

    # 최적 파라미터와 최고 정확도 출력
    print("최적 파라미터:", grid_search.best_params_)
    print("최고 정확도:", grid_search.best_score_)

(1-1) LGBM + 오버샘플링 x


In [82]:
lr_clf = LogisticRegression()
lo_grid(model=lr_clf,x=X_train,y=y_train)

최적 파라미터: {'C': 1}
최고 정확도: 0.7039841355687759


In [83]:
lr_clf = LogisticRegression(C=1)
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[714 172]
 [259 242]]
정확도: 0.6893, 정밀도: 0.5845, 재현율: 0.4830,    F1: 0.5290, AUC:0.6445


(1-2) LGBM + 오버샘플링O

In [84]:
lr_clf = LogisticRegression()
lo_grid(model=lr_clf,x=X_train_over,y=y_train_over)

최적 파라미터: {'C': 1}
최고 정확도: 0.7176121930567315


In [85]:
lr_clf = LogisticRegression(C=1)
get_model_train_eval(lr_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

오차 행렬
[[598 288]
 [132 369]]
정확도: 0.6972, 정밀도: 0.5616, 재현율: 0.7365,    F1: 0.6373, AUC:0.7057


### 2. LGBM 그리드서치

In [63]:
from sklearn.model_selection import GridSearchCV

def lgb_grid(model,x,y):

    # 그리드 서치 파라미터 그리드 정의
    param_grid = {
        'num_leaves': [10, 20, 30], # 트리가 가질 수 있는 최대 리프 노드 수
        'max_depth': [5, 10, 15], # 각 트리의 최대 깊이를 제한
        'learning_rate': [0.01, 0.1, 0.2], # 각 반복에서 모델 파라미터를 조정하는 정도를 제어
        'n_estimators': [50, 100, 200] # 트리의 개수, 즉 부스팅 라운드 수를 지정
    }

    # GridSearchCV를 사용하여 그리드 서치 수행
    grid_search = GridSearchCV(estimator=lgbm_clf, param_grid=param_grid, cv=3, scoring='accuracy')
    grid_search.fit(x, y)

    # 최적 파라미터와 최고 정확도 출력
    print("최적 파라미터:", grid_search.best_params_)
    print("최고 정확도:", grid_search.best_score_)

(2-1) LGBM + 오버샘플링X 결과

- 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'num_leaves': 20}


- 최고 정확도: 0.7157021813592933

In [ ]:
lgbm_clf = LGBMClassifier()
lgb_grid(model=lgbm_clf, x=X_train, y=y_train)

In [71]:
grid_lgbm = LGBMClassifier(learning_rate= 0.1, max_depth= 5, n_estimators= 50, num_leaves= 20)
get_model_train_eval(grid_lgbm, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2004, number of negative: 3543
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1224
[LightGBM] [Info] Number of data points in the train set: 5547, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.361276 -> initscore=-0.569829
[LightGBM] [Info] Start training from score -0.569829
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

(2-2) LGBM + 오버샘플링 결과


- 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 30}


- 최고 정확도: 0.7533163985323172

In [ ]:
lgbm_clf = LGBMClassifier()
lgb_grid(model=lgbm_clf, x=X_train_over, y=y_train_over)

In [70]:
grid_lgbm = LGBMClassifier(learning_rate= 0.1, max_depth= 10, n_estimators= 200, num_leaves= 30)
get_model_train_eval(grid_lgbm, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3543, number of negative: 3543
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2297
[LightGBM] [Info] Number of data points in the train set: 7086, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
오차 행렬
[[649 237]
 [192 309]]
정확도: 0.6907, 정밀도: 0.5659, 재현율: 0.6168,    F1: 0.5903, AUC:0.6746


### 그리드 서치를 통한 모델 선정
- 로지스틱 + 오버샘플링 = 정확도: 0.6972, 정밀도: 0.5616, 재현율: 0.7365,    F1: 0.6373, AUC:0.7057
- LGBM + 오버샘플링 = 정확도: 0.6907, 정밀도: 0.5659, 재현율: 0.6168,    F1: 0.5903, AUC:0.6746


## 테스트셋에 적합

In [92]:
grid_lgbm = LGBMClassifier(learning_rate= 0.1, max_depth= 10, n_estimators= 200, num_leaves= 30)
grid_lgbm.fit(X_train_over,y_train_over)
y_pred = grid_lgbm.predict(test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3543, number of negative: 3543
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000742 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2297
[LightGBM] [Info] Number of data points in the train set: 7086, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [95]:
sub = pd.read_csv('/content/drive/MyDrive/흡연자분류/sample_submission.csv')
sub['label'] = y_pred

In [98]:
sub.to_csv('/content/drive/MyDrive/흡연자분류/submission1.csv',index=False)